In [1]:
import datetime
import tweepy
import config
from tweepy import OAuthHandler
import json
import pandas as pd
import csv
import os
import time
import glob
import re
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import preprocessor as p
import numpy as np
words = set(nltk.corpus.words.words())
sw=set(stopwords.words('english'))

In [2]:
# Set connection to Tweepy with academic account. I've put my API keys in a .py file called API_keys.py
from my_api_keys import bearer_token
tweepyclient = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [3]:
# Upload all local csv files to single pandas dataframe

path = 'data/' # use your path

trail_files = glob.glob(os.path.join(path, "*trailrunning.csv"))
climate_files = glob.glob(os.path.join(path, "*climate.csv"))

trailtweets = pd.concat((pd.read_csv(f) for f in trail_files), ignore_index=True)
climatetweets = pd.concat((pd.read_csv(f) for f in climate_files), ignore_index=True)

In [4]:
trail_files

['data/20221110_000000_tweets_trailrunning.csv']

In [5]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # remove mentions
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # remove hashtags
    temp = re.sub(r'http\S+', '', temp)  # remove links
    temp = re.sub('[()!?]', ' ', temp)  # remove punct
    temp = re.sub('\[.*?\]',' ', temp)  # remove punct
    temp = re.sub("[^a-z0-9]"," ", temp) # remove non-alphanumeric
    temp = temp.split()  # tokenize
    temp = [w for w in temp if w not in sw]  # remove stopwords
    temp = " ".join(word for word in temp)  # rejoin into tweets again
    return temp

In [6]:
trailtweets['fulltext_clean'] = trailtweets['text'].apply(clean_tweet)  # create new column of cleaned tweets

In [7]:
test = trailtweets.head(3)

In [8]:
test

,user_id,username,follower_count,total_tweets,description,location,tweet_id,text,created_at,retweets_count,replies_count,likes_count,quote_count,fulltext_clean
0,1493747747115405313,JunJP0903,944,10694,男性\n公立中学校教諭→塾経営→NPO法人代表理事→現在サイト運営\n\nオリジナルアイテム...,"Nishinomiya-shi, Hyogo",1590131096196624384,RT @meesuke1: #PrayForWorldPeace 🍀🌏\n#イマソラ #いま...,2022-11-08 23:56:11+00:00,11,0,0,0,rt 6 127 4 4km 179kcal 57
1,22719296,B3toRamirez,532,36116,ESL/EFL Teacher 💙🐺 Hincha de @Macara_Oficial. ...,"Ambato, Ecuador",1590130296208650240,RT @EfrenIcaza: @Ladrillazo Normalmente los so...,2022-11-08 23:53:00+00:00,4,0,0,0,rt normalmente los solitarios escogemos el el ...
2,1131468540,Dinsdale81,384,12012,"Ipswich Town FC 🚜🚜\n40, Anglia.\nTraining for ...",Suffolk,1590130167623450624,Wet run today. Ran an old trail route that I u...,2022-11-08 23:52:29+00:00,0,0,1,0,wet run today ran old trail route used maratho...


In [23]:
# test with just JC's handle

handle = "37chandler"
user_obj = tweepyclient.get_user(username=handle)

followers = tweepyclient.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

In [24]:
num_to_print = 3

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break

Lexi lists 'None' as their location.
 Following: 450, Followers: 27.

Dave Renn lists 'None' as their location.
 Following: 69, Followers: 10.

Lionel lists 'None' as their location.
 Following: 200, Followers: 201.



In [29]:
for user in test['username']:
    
    followers_data = []
    
    user_obj = tweepyclient.get_user(username=user)
    print(user_obj)

Response(data=<User id=1493747747115405313 name=JUN username=JunJP0903>, includes={}, errors=[], meta={})
Response(data=<User id=22719296 name=Beto Ramírez Escobar username=B3toRamirez>, includes={}, errors=[], meta={})
Response(data=<User id=1131468540 name=Dinsdale username=Dinsdale81>, includes={}, errors=[], meta={})


In [36]:
num_followers_to_pull = 100 # feel free to use this to limit the number of followers you pull.

for user in test['username']:
    
    followers_data = []
    
    user_obj = tweepyclient.get_user(username=user)

    for response in tweepy.Paginator(tweepyclient.get_users_followers, user_obj.data.id,
                                     max_results=1000, limit=num_followers_to_pull,
                                     user_fields=["id"]):
        
        followers_data.append(response)
        
    #test[user].append(followers_data)
        


1